# Process Receiver Functions

This notebook does the main processing of the measured receiver functions.
The processing includes

1. Quality filtering
2. Measuring of $P$-to-$S$ sediment basement conversion delay time $t_{PS_b}$
3. Measuring of the two-way travel-time of shear waves in the sediment layer $t_2$

In [ ]:
import rf
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os
from obspy.clients.fdsn import Client
from obspy.core import UTCDateTime
from datetime import datetime
from collections import defaultdict
import pygmt
import random
from joblib import Parallel, delayed
from scipy.signal import argrelmax

from utils import australia_basemap

import warnings

warnings.filterwarnings("ignore")

## Load RFs and QC

In [ ]:
try:
    dataroot = os.environ["DATADIR"]
except KeyError:
    dataroot = os.path.join("..", "data","rf_data")

In [ ]:
def quality_filter(stream: rf.RFStream) -> rf.RFStream:
    """
    Applies various final quality controls to the RFs
    """
    rf_station_dict = defaultdict(rf.RFStream)
    for trace in stream:
        # Only keep traces where largest arrival is positive
        if trace[np.argmax(np.abs(trace.data))] < 0:
            continue

        # Only keep traces where largest arrival is within 2 seconds after direct P
        largest_arrival = trace.times()[np.argmax(trace.data)] - (
            trace.stats.onset - trace.stats.starttime
        )
        if largest_arrival > 2 or largest_arrival < 0:
            continue

        rf_station_dict[trace.stats.station] += rf.RFStream([trace])

    # Only keep stations with 10 or more traces
    rf_station_dict = {
        k: v.sort(["back_azimuth"]) for k, v in rf_station_dict.items() if len(v) >= 10
    }

    return sum(rf_station_dict.values(), start=rf.RFStream())


def load_network_rfs(network: str):
    """
    Loads the rfs for a network and performs some initial quality filtering
    Returns a stream for the network that has been trimmed and moveout corrected and a count of how many stations and traces were recorded BEFORE any quality control
    """
    network_dir = os.path.join(dataroot, f"{network}-analysis")
    try:
        latest_run = max(
            [
                os.path.join(network_dir, d)
                for d in os.listdir(network_dir)
                if os.path.isdir(os.path.join(network_dir, d)) and d != "corrections"
            ],
            key=os.path.getmtime,
        )
    except ValueError as e:
        print(f"No run found for network {network}.")
        return (network, 0, 0)

    # Grab the latest .h5 file - should be the outputs of qc
    try:
        h5_file = max(
            [
                os.path.join(latest_run, f)
                for f in os.listdir(latest_run)
                if os.path.splitext(f)[1] == ".h5"
            ],
            key=os.path.getmtime,
        )
    except ValueError as e:
        print(f"No run found for network {network}.")
        return (network, 0, 0)

    try:
        stream = rf.read_rf(h5_file, format="h5")
    except:
        print(f"Something went wrong when reading {h5_file}. Moving on...")
        return (network, 0, 0)
    # Drop transverse component
    stream = stream.select(channel="??R")

    # Trim to a reasonable time length
    starttime = -5
    endtime = 10
    stream = stream.trim2(starttime, endtime, reftime="onset")

    stream.moveout()

    # Count initial number of stations and traces
    nstations = len(set([trace.stats.station for trace in stream]))
    ntraces = len(stream)

    # Apply quality filter
    stream = quality_filter(stream)
    if len(stream) == 0:
        print(f"Nothing left after quality control for network {network}. Moving on...")
        return (network, nstations, ntraces)

    return (stream, nstations, ntraces)

Load all the networks in parallel

In [ ]:
networks = [
    net[:2]
    for net in os.listdir(dataroot)
    if len(net.split("-")[0]) == 2 and net.split("-")[1] == "analysis"
]

result = Parallel(n_jobs=-1)(delayed(load_network_rfs)(net) for net in networks)
rfstream = rf.RFStream()  # Stream with every RF
init_nstations = {}
init_ntraces = {}
for res in result:
    _netstream, _nstations, _ntraces = res
    if isinstance(_netstream, rf.RFStream):
        net = _netstream[0].stats.network
        rfstream += _netstream
    elif isinstance(_netstream, str):
        net = _netstream
    init_nstations[net] = _nstations
    init_ntraces[net] = _ntraces
del result

Remove traces with exeedingly large amplitudes that will dominate the stacks

In [ ]:
log10_amp_max = np.array([tr.stats.log10_amp_max for tr in rfstream])
mean = np.mean(log10_amp_max)
stddev = np.std(log10_amp_max)
normed = (log10_amp_max - mean) / stddev
amplitude_threshold = 4  # in units of stddev
indices = np.where(np.abs(normed) <= amplitude_threshold)[0]
rfstream = rf.RFStream(map(rfstream.__getitem__, indices))

Count remaining stations and traces

In [ ]:
empty_networks = []
for net in networks:
    stream = rfstream.select(network=net)
    nstations = len(set([trace.stats.station for trace in stream]))
    ntraces = len(stream)
    print(f"{net}: {nstations}/{init_nstations[net]} stations kept after quality control")
    print(f"{net}: {ntraces}/{init_ntraces[net]} radial RFs kept after quality control")
    if nstations == 0 or ntraces == 0:
        empty_networks.append(net)
for empty in empty_networks:
    networks.remove(empty)

In [ ]:
print(f"Networks {', '.join(empty_networks)} have no more traces after quality control")

## Measure delay time

In [ ]:
def get_tpsb(trace: rf.rfstream.RFTrace):
    """
    Finds the time of the first local maximum of trace
    """
    ind = argrelmax(trace.data)[0][0]
    return trace.times()[ind] - (trace.stats.onset - trace.stats.starttime)

In [ ]:
def station_stack(stream: rf.RFStream) -> rf.RFStream():
    """
    TO BE USED ONLY WHEN STREAM ONLY CONTAINS TRACES FROM THE SAME STATION
    RFStream.stack() forces stacking for separate ids, including channels
    Access original obspy Stream.stack() to stack BHR and HHR channels
    Copies meta data as in RFStream.stack()
    """
    _stream = stream.copy()  # Stream.stack() operates in place
    stack = super(rf.RFStream, _stream).stack(group_by="{network}.{station}")
    assert len(stack) == 1
    tr = stream[0]
    data = stack[0].data
    header = {}
    for entry in (
        "network",
        "station",
        "sampling_rate",
        "phase",
        "moveout",
        "station_latitude",
        "station_longitude",
        "station_elevation",
        "processing",
    ):
        if entry in tr.stats:
            header[entry] = tr.stats[entry]
    _tr = rf.rfstream.RFTrace(data=data, header=header)
    if "onset" in tr.stats:
        onset = tr.stats.onset - tr.stats.starttime
        _tr.stats.onset = _tr.stats.starttime + onset
    return rf.RFStream(_tr)


def stack_by_station(stream: rf.RFStream) -> rf.RFStream():
    # Station dictionary
    rf_station_dict = defaultdict(rf.RFStream)
    for trace in stream:
        rf_station_dict[trace.stats.station] += rf.RFStream([trace])

    # stack stations individually, otherwise station coordinates also get stacked
    stacks = rf.RFStream()
    for v in rf_station_dict.values():
        stacks += station_stack(v)

    return stacks


def get_delays(stream: rf.RFStream) -> rf.RFStream():
    # Get delay of each stack
    for stack in stream:
        delay = get_tpsb(stack)
        if delay >= 0:
            stack.stats["delay"] = delay
        else:
            print(
                f"Negative delay obtained for {stack.stats.network}.{stack.stats.station}"
            )
            stream.remove(stack)

    return stream.sort(["delay", "network", "station"])

In [ ]:
rfstacks = stack_by_station(rfstream)  # stream with station stacks
rfstacks = get_delays(rfstacks)

Plot traces, stacks and maps

In [ ]:
def plot_map(stream: rf.RFStream, save_dir: str=".", show: bool=False):
    lats = np.array([trace.meta.station_latitude for trace in stream])
    lons = np.array([trace.meta.station_longitude for trace in stream])
    nets = np.array([trace.meta.network for trace in stream], dtype=str)
    delays = np.array([trace.meta.delay for trace in stream])

    fig, region, projection = australia_basemap()
    pygmt.makecpt(cmap="hot", truncate=[0, 0.8], series=[0, 1, 0.1],background="o", reverse=True)
    fig.plot(
        region=region,
        projection=projection,
        x=lons,
        y=lats,
        style=f"tc",
        fill=delays,
        cmap=True,
        size=np.full_like(lons, 0.25),
    )
    fig.colorbar(
        region=region,
        projection=projection,
        frame=["af+lDelay Time TPsb (s)", f"y+lmax={delays.max():.2f}"],
        position="JBC+ef",
    )
    if save_dir is not None:
        mapfile = os.path.join(save_dir, "delay_map.pdf")
        fig.savefig(mapfile)
    if show:
        fig.show()

In [ ]:
dry_run = False
if not dry_run:
    # Make output directories
    now = (datetime.now()).strftime(format="%Y%m%d_%H%M%S")
    processedroot = os.path.join(dataroot, "..", "processed")
    outdir = os.path.join(processedroot, now)

    for net in networks:
        os.makedirs(os.path.join(outdir, net))


    def plot_rfs_stacks_map(network, stream, stacks):
        outdir = os.path.join(processedroot, now, network)
        stations = set([stack.stats.station for stack in stacks])
        # plot RFs at each station
        for station in stations:
            rfs = stream.select(station=station)
            plot_name = os.path.join(outdir, f"{station}.pdf")
            fig = rfs.plot_rf(
                fname=plot_name,
                fig_width=4,
                fillcolors=("green", None),
                trace_height=0.1,
                scale=3,
                show_vlines=True,
                trim=(-2, 6),
            )  # this figure needs some bits to be moved around

        # Plot the stacked RFs of the network
        plot_name = os.path.join(outdir, f"stacks.pdf")
        fig = stacks.plot_rf(
            fname=plot_name,
            fig_width=4,
            fillcolors=("green", None),
            trace_height=0.1,
            scale=3,
            show_vlines=True,
            trim=(-2, 6),
            info=(),
        )  # customise this plot to use a colourmap

        # Plot map of network delay times
        plot_map(stacks, outdir)


    # Plots for each network
    Parallel(n_jobs=-1)(
        delayed(plot_rfs_stacks_map)(
            net, rfstream.select(network=net), rfstacks.select(network=net)
        ) for net in networks
    )

    # Plot the full map
    plot_map(rfstacks, outdir, show=True)

Save the delay times in txt file

In [ ]:
if not dry_run:
    with open(os.path.join(outdir, "delays.txt"), "w") as f:
        for stack in rfstacks:
            f.write(
                f"{stack.meta.network:4}\t{stack.meta.station:6}\t{stack.meta.station_longitude}\t{stack.meta.station_latitude}\t{stack.stats.delay:.3f}\n"
            )